In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
path = "/data/pcpe_01.csv"
df = pd.read_csv(path, delimiter=';')
df

,NUMERO_CASO,NUMERO_BANCO,NUMERO_AGENCIA,NUMERO_CONTA,TIPO,CPF_CNPJ_TITULAR,CNAB,CNAB=101,CNAB=119,CNAB=120,...,NUMERO_AGENCIA_OD,NUMERO_CONTA_OD,I-a,I-b,I-c,I-d,I-e,DIA_LANCAMENTO,MES_LANCAMENTO,ANO_LANCAMENTO
0,CASO-01-L1K4U4,O6Y,1965,153602,2,ZS9104223348,218,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,2017
1,CASO-01-L1K4U4,O6Y,1965,153602,2,ZS9104223348,218,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,2017
2,CASO-01-L1K4U4,L6X,742,810886,2,RJ3341718546,211,0,0,0,...,75,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2017
3,CASO-01-L1K4U4,L6X,742,810886,2,RJ3341718546,211,0,0,0,...,75,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2017
4,CASO-01-L1K4U4,W5Y,554,902093,1,IK6356601049,201,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138300,CASO-01-L1K4U4,O9A,490,296083,1,KO2307484082,206,0,0,0,...,490,296083.0,NaN,NaN,NaN,NaN,NaN,1,3,2021
138301,CASO-01-L1K4U4,O9A,490,296083,1,KO2307484082,102,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,1,3,2021
138302,CASO-01-L1K4U4,O9A,490,296083,2,KO2307484082,102,0,0,0,...,490,296083.0,NaN,NaN,NaN,NaN,NaN,1,3,2021
138303,CASO-01-L1K4U4,O6Y,5776,191834,1,KO2307484082,205,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,1,3,2021


In [5]:
columns_num = [
    'DIA_LANCAMENTO', 
    'MES_LANCAMENTO',
    'VALOR_TRANSACAO',
    'VALOR_SALDO',
]

columns_cat = [
    'TIPO',
    'CNAB',
    'NATUREZA_SALDO'
]

target = ['I-d']
y = df[target].values
y = np.nan_to_num(y)
y

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [6]:
from mlex import CompositeTranformer
tranformer = CompositeTranformer(
    numeric_features=columns_num,
    categorical_features=columns_cat
)
Xt = tranformer.encoder.fit_transform(df)

2024-05-09 10:45:17.763465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
from mlex.utils.splits import PastFutureSplit

split = PastFutureSplit()

X_train, X_test, y_train, y_test = split.train_test_split(Xt,y)

In [8]:
from mlex import SequenceTransfomer
p = SequenceTransfomer()
X_train = np.array(X_train)
y_train = np.array(y_train)
data_train = p.transform(
    X = X_train,
    y = y_train
)

X_test = np.array(X_test)
y_test = np.array(y_test)
data_test = p.transform(
    X = X_train,
    y = y_train
)





2024-05-09 10:45:21.691093: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-05-09 10:45:21.691136: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: a1c211f74e0b
2024-05-09 10:45:21.691147: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: a1c211f74e0b
2024-05-09 10:45:21.691285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2024-05-09 10:45:21.691319: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 530.30.2


In [9]:
import keras
import keras.optimizers.adam

In [10]:
from mlex.pipelines.simple_pipeline import SimplePipeline
from mlex.models.models import SimpleRNNModel

my_model = SimpleRNNModel(input_shape=Xt.shape)

my_model.get_model()


In [11]:
my_model.compile()

In [12]:

my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 138305, 16)        992       
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 16)                528       
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 1,537
Trainable params: 1,537
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = my_model.fit(data_train)

Epoch 1/10


2024-05-09 10:45:27.409741: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [69143]
	 [[{{node Placeholder/_8}}]]
2024-05-09 10:45:27.410468: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [69152,45]
	 [[{{node Placeholder/_0}}]]


541/541 [==============================] - 3s 4ms/step - loss: 0.2954 - acc: 0.8899 - auc: 0.7807
Epoch 2/10
541/541 [==============================] - 1s 3ms/step - loss: 0.2535 - acc: 0.8985 - auc: 0.8588
Epoch 3/10
541/541 [==============================] - 2s 3ms/step - loss: 0.2403 - acc: 0.9022 - auc: 0.8771
Epoch 4/10
541/541 [==============================] - 1s 3ms/step - loss: 0.2311 - acc: 0.9063 - auc: 0.8879
Epoch 5/10
541/541 [==============================] - 2s 3ms/step - loss: 0.2244 - acc: 0.9097 - auc: 0.8947
Epoch 6/10
541/541 [==============================] - 2s 3ms/step - loss: 0.2175 - acc: 0.9132 - auc: 0.9018
Epoch 7/10
541/541 [==============================] - 1s 3ms/step - loss: 0.2097 - acc: 0.9170 - auc: 0.9093
Epoch 8/10
541/541 [==============================] - 1s 2ms/step - loss: 0.2039 - acc: 0.9199 - auc: 0.9141
Epoch 9/10
541/541 [==============================] - 1s 3ms/step - loss: 0.1992 - acc: 0.9227 - auc: 0.9184
Epoch 10/10
541/541 [=========

In [14]:
y_pred = my_model.predict(data_test)

2024-05-09 10:45:45.070545: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [69143]
	 [[{{node Placeholder/_8}}]]
2024-05-09 10:45:45.071386: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [69143]
	 [[{{node Placeholder/_8}}]]


541/541 [==============================] - 2s 3ms/step


In [15]:
np.quantile(y_pred, 0.95)

0.7575871586799621